In [ ]:
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
# Function for recording data
def collect_data(duration=5):
    """Collects log data from the cable robot.  Returns tuple of Nx4 arrays for lengths and vels"""
    with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port='/dev/tty.usbmodem103568503') as robot:
        tstart = time.time()
        while True:
            robot.update()
            time.sleep(0.01)
            if (time.time() - tstart > duration):
                break
        robot.send('d10,100')
    motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
    motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
    xy = np.array([[dat['controller'].cur_x, dat['controller'].cur_y] for dat in robot.all_data])
    xy_com = np.array([[dat['controller'].set_x, dat['controller'].set_y] for dat in robot.all_data])
    return motor_ls, motor_ldots, xy, xy_com
def collect_points(N):
    motor_ls = []
    motor_ldots = []
    xy = []
    xy_commanded = []
    for i in range(N):
        input(f'Press enter to record point {i}')
        a, b, c, d = collect_data(0.5)
        motor_ls.append(a[-1])
        motor_ldots.append(b[-1])
        xy.append(c[-1])
        xy_commanded.append(d[-1])
        print(f'point {i}: ', motor_ls[-1], motor_ldots[-1], xy[-1], xy_commanded[-1])
    return np.array(motor_ls), np.array(motor_ldots), np.array(xy), np.array(xy_commanded)


In [ ]:
collect_points(1)

In [ ]:
# Collect 4 corners, in order: BR, TR, TL, BL
# For each of the 4 corners, put the brush in "painting" position and just barely touch the mullions with the brush.
USE_CORNERS = True
if True:
    # Collect some data
    ls4, ldots4, xy4, xy_com4 = collect_points(4)
    print(ls4)
    print(ldots4)
    print(xy4)
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/4pts_data.npz', ls4=ls4, ldots4=ldots4, est4=xy4, com4=xy_com4)
    np.savez(f'/tmp/4pts_data_{time.time()}.npz', ls4=ls4, ldots4=ldots4, est4=xy4, com4=xy_com4)
else:
    with np.load('/tmp/4pts_data.npz') as f:
        ls4 = f['ls4']
        ldots4 = f['ldots4']
        xy4 = f['est4']
        xy_com4 = f['com4']

# print(ls4)
# print(ldots4)
# print(xy4)
# xy4 = xy_com4

In [ ]:
# Collect 2 corners on the left-side of the right-pane only.  TL then BL
if True:
    # Collect some data
    ls2, ldots2, xy2, xy_com2 = collect_points(2)
    print(ls2)
    print(ldots2)
    print(xy2)
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/2pts_data.npz', ls2=ls2, ldots2=ldots2, est2=xy2, com2=xy_com2)
    np.savez(f'/tmp/2pts_data_{time.time()}.npz', ls2=ls2, ldots2=ldots2, est2=xy2, com2=xy_com2)

    xy4_bak = xy4.copy()

In [ ]:
# Collect 2 corners on the right-side of the left-pane only.  TR then BR
if True:
    # Collect some data
    ls2l, ldots2l, xy2l, xy_com2l = collect_points(2)
    print(ls2l)
    print(ldots2l)
    print(xy2l)
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/2lpts_data.npz', ls2l=ls2l, ldots2l=ldots2l, est2l=xy2l, com2l=xy_com2l)
    np.savez(f'/tmp/2lpts_data_{time.time()}.npz', ls2l=ls2l, ldots2l=ldots2l, est2l=xy2l, com2l=xy_com2l)

    # xy4_bak = xy4.copy()

In [ ]:
PANE = 'CENTER_BOTH'
PANE = 'LEFT_MID_HALF'
PANE = 'MIDDLE_BOTH'
PANE = 'SPLIT_RIGHT_ONLY'
PANE = 'SPLIT_LEFT_3'
PANE = 'CENTER_BOTH_ALT'
PANE = 'LEFT_MID_ALT'

if PANE == 'SPLIT_RIGHT_ONLY':
    xy4 = np.vstack((xy4_bak[0], xy4_bak[1], xy2[0], xy2[1]))
    PANE = 'RIGHT_MID_HALF'
if PANE == 'SPLIT_LEFT_3':
    xy4 = np.vstack((xy2[1], xy2[0], xy4_bak[2], xy4_bak[3]))
if PANE == 'CENTER_BOTH_ALT':
    xy4 = np.vstack((xy2[1], xy2[0], xy2l[0], xy2l[1]))
if PANE == 'LEFT_MID_ALT':
    xy4 = np.vstack((xy2l[1], xy2l[0], xy4_bak[2], xy4_bak[3]))
    PANE = 'LEFT_MID_HALF'

In [ ]:
# 4 Corner Constants
# First define X1, X2, Y1, Y2 are the bounds of the pane relative to the bottom-left pulley.
inch2m = lambda x: x * 0.0254
if PANE == 'CENTER_BOTTOM':
    PULLEY_O = .037
    X1 = 1.887 - PULLEY_O
    X2 = X1 + inch2m(84)
    Y1 = 5.417 - 4.614
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'CENTER_TOP':
    PULLEY_O = .037
    X1 = 1.887 - PULLEY_O
    X2 = X1 + inch2m(84)
    Y1 = 5.417 - 4.614 + inch2m(74)
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'CENTER_BOTH':
    PULLEY_O = .037
    X1 = 1.887 - PULLEY_O
    X2 = X1 + inch2m(84)
    Y1 = 5.417 - 4.614
    # FUDGE = 6e-2
    FUDGE = 0
    Y2 = Y1 + inch2m(71.5 * 2 + 2.5) + FUDGE
    # Fudge
    Y1 -= 4e-2
elif PANE == 'LEFT_MID_HALF':
    X1 = 1.887 - PULLEY_O - inch2m(3) - inch2m(35.5)
    X2 = X1 + inch2m(35.5)
    Y1 = 5.417 - 4.614 + inch2m(71.5/2) + inch2m(2.5/2)
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'RIGHT_MID_HALF':
    X1 = 1.887 - PULLEY_O - inch2m(3) - inch2m(35.5)
    X1 += inch2m(35.5) + inch2m(3) + inch2m(84) + inch2m(3)
    X2 = X1 + inch2m(35.5)
    Y1 = 5.417 - 4.614 + inch2m(71.5/2) + inch2m(2.5/2)
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'MIDDLE_BOTH':
    X1 = 1.887 - PULLEY_O - inch2m(3) - inch2m(35.5)
    X2 = X1 + inch2m(35.5) + inch2m(3) + inch2m(84) + inch2m(3) + inch2m(35.5)
    Y1 = 5.417 - 4.614 + inch2m(71.5/2) + inch2m(2.5/2)
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'SPLIT_LEFT_3':
    X1 = 1.887 - PULLEY_O - inch2m(3) - inch2m(35.5)
    X2 = X1 + inch2m(35.5) + inch2m(3) + inch2m(84) + inch2m(3) + BRUSH_R * 2
    Y1 = 5.417 - 4.614 + inch2m(71.5/2) + inch2m(2.5/2)
    Y2 = Y1 + inch2m(71.5)
elif PANE == 'CENTER_BOTH_ALT':
    X = 1.887 - PULLEY_O - inch2m(3)
    X1 = X - BRUSH_R * 2
    X2 = X + inch2m(3) + inch2m(84) + inch2m(3) + BRUSH_R * 2
    Y1 = 5.417 - 4.614 + inch2m(71.5/2) + inch2m(2.5/2)
    Y2 = Y1 + inch2m(71.5)
else:
    raise RuntimeError(f'Unknown pane: {PANE}')

print(f'pane: {PANE}, {X1 = }, {X2 = }, {Y1 = }, {Y2 = }')
print(f'sanity check: pane width (84"=2.1336m): {X2 - X1 = }')
if 'CENTER' in PANE:
    print(f'              total width (5.833m): {X2 + X1 = }')

BRUSH_R = (3.1e-2 + 0.3e-2) / 2 # 3.1cm brush diameter, 0.3cm fuzz factor
baseTbrush = np.array([21.0e-2, 38.5e-2])       # brush in the base coordinate frame
baseTcdprrest = -np.array([0.5334, 0.381]) / 2  # cdpr has a carriagewidth/2 offset we need to correct for
XY4_ORIGIN = np.array([[X2 - BRUSH_R, Y1 + BRUSH_R],
                [X2 - BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y1 + BRUSH_R]]) - baseTbrush
print('Corner Locs: (using bottom-left coordinate system)')
print(XY4_ORIGIN)

print('Expected Corner Locs: (using carriage-center coordinate system)')
XY4 = XY4_ORIGIN - baseTcdprrest
print(XY4)
print('Actual corner locs: (using carriage-center coordinate system)')
print(xy4)
print('60mm buffer limits: (using carriage-center coordinate system)')
XY60 = np.array([[X2 - 0.06, Y1 + 0.06],
                 [X2 - 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y1 + 0.06]]) - baseTbrush - baseTcdprrest
print(XY60)
s = f'xLl{XY60[2][0]};xLr{XY60[0][0]};xLd{XY60[0][1]};xLu{XY60[1][1]}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')

In [ ]:
# Compute an xy homography correction using the 4 corners
def compute_H(act, est):
    # act is 2x4, est is 2x4
    A = np.zeros((8, 9))
    for i in range(4):
        A[2*i, 0:3] = act[:, i]
        A[2*i, 6:9] = -est[0, i] * act[:, i]
        A[2*i+1, 3:6] = act[:, i]
        A[2*i+1, 6:9] = -est[1, i] * act[:, i]
    _, _, V = np.linalg.svd(A)
    H = V[-1, :].reshape(3, 3)
    return H

def make_homogeneous(x):
    return np.vstack((x, np.ones(x.shape[1])))

In [ ]:
H = compute_H(make_homogeneous(XY4.T), make_homogeneous(xy4.T))
print(H)

# Sanity check
def apply_H(good):
    pred = H @ make_homogeneous(good.T)
    pred /= pred[-1, :]
    return pred[:-1, :].T
print('applying homography to expected corners: ', apply_H(XY4))
print('actual measure corners:                  ', xy4)

In [ ]:
# Here are the limits we should use post-homography
print('60mm buffer limits:')
XY60_cdpr_coords = apply_H(XY60)
print(XY60_cdpr_coords)
xmin, xmax = min(XY60_cdpr_coords[:, 0]), max(XY60_cdpr_coords[:, 0])
ymin, ymax = min(XY60_cdpr_coords[:, 1]), max(XY60_cdpr_coords[:, 1])
s = f'xLl{xmin};xLr{xmax};xLd{ymin};xLu{ymax}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')
print('The following should be typed into gcode webpage:')
print('H:')
print(H.tolist())
print('Expected Mural Translation:')
print(np.min(XY60, axis=0))
print('Expected Mural Dimensions (with the given scale factor, this should be the total width & height):')
print(np.max(XY60, axis=0) - np.min(XY60, axis=0))
# print((np.max(XY60, axis=0) - np.min(XY60, axis=0)) + 0.12)

In [ ]:
xy4

In [ ]:
plt.plot(*XY60.T, 'r*')
plt.plot(*XY60_cdpr_coords.T, 'k*')
plt.axis('equal');

In [ ]:
# Do a test position.
xy = 3, 0.66850 # bottom panel, bottom-center
xy = 3, 2.36460 # bottom panel, top-center
xy = 3, 2.54810 # top panel, bottom-center
# xy = 3, 4.24420 # top panel, top-center
xy = 0.98880, 3.30440 # left panel, left-top
apply_H(np.array(xy)[None, :])

In [ ]:
# All Left Corners
print('Left Corners:')
X1, X2, Y1, Y2 = 0.98880, 1.77050, 1.60830, 3.30440
print(apply_H(np.array([[X2, Y1], [X2, Y2], [X1, Y2], [X1, Y1]])))
# All Right Corners
print('Right Corners:')
X1, X2, Y1, Y2 = 4.17650, 4.95820, 1.60830, 3.30440
print(apply_H(np.array([[X2, Y1], [X2, Y2], [X1, Y2], [X1, Y1]])))
# All Bottom Corners
print('Bottom Corners:')
X1, X2, Y1, Y2 = 1.96670, 3.98030, 0.66850, 2.36460
print(apply_H(np.array([[X2, Y1], [X2, Y2], [X1, Y2], [X1, Y1]])))
# All Top Corners
print('Top Corners:')
X1, X2, Y1, Y2 = 1.96670, 3.98030, 2.54810, 4.24420
print(apply_H(np.array([[X2, Y1], [X2, Y2], [X1, Y2], [X1, Y1]])))

In [ ]:

Bottom Corners:
[[3.97431734 0.62439224]
 [3.96409959 2.37976267]
 [1.91243231 2.3490364 ]
 [1.9024599  0.58422505]]
Top Corners:
[[3.96300603 2.56763153]
 [3.95300595 4.28560728]
 [1.92325842 4.26492904]
 [1.91349955 2.5379051 ]]

In [ ]:
# To test the gcode homography, for the first
X, Y = 1355.851/1e3, 12.000/1e3
print('Original: ', X, Y)
X, Y = (X * 0.94379391 + 1.9667), (Y *   + 0.6685)
print(X, Y)
print('To be sent to the robot:')
apply_H(np.array([[X, Y]]).reshape(1, 2))